In [1]:
import sys
import os
import face_recognition
import cv2
import dlib
import numpy as np
from datetime import datetime, timedelta
import csv
import subprocess
import fil1
import os
from scipy.spatial import distance as dist




def run_registration_program():
    registration_program = "python3 register.py"  # Replace with the command to run your register.py program
    subprocess.Popen(registration_program, shell=True)
# Define constants
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3

# Initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # Compute the euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # Return the eye aspect ratio
    return ear

# Initialize dlib face detector and facial landmark predictor
def run_recognition():
    
    file_path = '/home/parthjoshi/Desktop/webside/face_detection/Face_recognition/shape_predictor_68_face_landmarks.dat'

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(file_path)
   

    # Initialize lists for known face encodings and names
    previous_landmarks = None
    liveness_threshold = 5
    known_faces = []
    known_names = []

    # Directory containing face images
    directory = fil1.faces_directory  # Replace with the path to your directory

    # Load known faces and encodings from directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            name = os.path.splitext(filename)[0]
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)

            # Detect face landmarks
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb)
            if len(face_locations) > 0:
                face_location = face_locations[0]
                top, right, bottom, left = face_location

                # Perform face encoding
                face_encoding = face_recognition.face_encodings(rgb, [face_location])[0]

                # Add face encoding and name to the lists
                known_faces.append(face_encoding)
                known_names.append(name)

    # Initialize video capture
    cap = cv2.VideoCapture(0)  # Replace with your video source if not using the webcam

    # Get screen resolution
    screen_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    screen_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # Calculate ROI dimensions
    roi_width = int(screen_width / 2)
    roi_height = int(screen_height / 2)
    roi_x = int((screen_width - roi_width) / 2)
    roi_y = int((screen_height - roi_height) / 2)

    # Initialize tolerance for face recognition (confidence factor)
    tolerance = 0.5  # Adjust this value as needed

    # Initialize dictionary to track last detected time for recognized faces and unregistered faces
    recognized_last_detected = {}
    unregistered_last_detected = {}

    # Create resizable window
    cv2.namedWindow("Face Recognition", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Face Recognition", int(screen_width), int(screen_height))

    # Initialize variables for motion detection
    prev_frame_gray = None
    stationary_counter = 0
    
    # Eye aspect ratio thresholds and counters
    EYE_AR_THRESH = 0.2
    EYE_AR_CONSEC_FRAMES = 3
    COUNTER = 0
    TOTAL_BLINKS = 0

    # Main loop
    while True:
        # Read a frame
        ret, frame = cap.read()

        # Convert frame to gray for motion detection
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        frame_gray = cv2.GaussianBlur(frame_gray, (21, 21), 0)

        # Initialize previous frame if it's the first iteration
        if prev_frame_gray is None:
            prev_frame_gray = frame_gray
            continue

        # Calculate absolute difference between current and previous frames
        frame_delta = cv2.absdiff(prev_frame_gray, frame_gray)

        # Apply threshold to detect significant intensity differences
        threshold = 30
        _, frame_threshold = cv2.threshold(frame_delta, threshold, 255, cv2.THRESH_BINARY)

        # Dilate the thresholded image to fill in holes
        kernel = np.ones((5, 5), np.uint8)
        frame_threshold = cv2.dilate(frame_threshold, kernel, iterations=2)

        # Find contours of significant differences
        contours, _ = cv2.findContours(frame_threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Convert frame back to RGB for face recognition
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Draw ROI on the frame
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 0, 255), 2)

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(rgb)
        face_encodings = face_recognition.face_encodings(rgb, face_locations)

        # Initialize set to track unregistered face IDs
        unregistered_faces = set()

        # Iterate over the face encodings
        for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
            # Check if the face is completely outside the ROI
            if right < roi_x or left > roi_x + roi_width or bottom < roi_y or top > roi_y + roi_height:
                # Draw bounding box in gray color
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                continue

            # Compare face encodings with known faces
            face_distances = face_recognition.face_distance(known_faces, face_encoding)
            best_match_index = np.argmin(face_distances)
            face_distance = face_distances[best_match_index]

            if face_distance <= tolerance:
                name = known_names[best_match_index]
                current_time = datetime.now()

                if name not in recognized_last_detected or (current_time - recognized_last_detected[name]) >= timedelta(minutes=2):
                    recognized_last_detected[name] = current_time
                    record_attendance(name, current_time)
                else:
                    unregistered_faces.add(name)
                
                landmarks = predictor(rgb, dlib.rectangle(left, top, right, bottom))
                
                landmarks_np = np.array([[p.x, p.y] for p in landmarks.parts()])

                # Get the coordinates for the left and right eye
                leftEye = landmarks_np[36:42]
                rightEye = landmarks_np[42:48]

                # Compute the eye aspect ratio for both eyes
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)

                # Average the eye aspect ratio together for both eyes
                ear = (leftEAR + rightEAR) / 2.0

                # Check to see if the eye aspect ratio is below the blink threshold
                if ear < EYE_AR_THRESH:
                    COUNTER += 1
                else:
                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        TOTAL_BLINKS += 1
                    COUNTER = 0

                # Draw the computed eye aspect ratio and the number of blinks on the frame
                cv2.putText(frame, "Blinks: {}".format(TOTAL_BLINKS), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                #cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                #cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
            else:
                # Draw bounding box for unknown face in gray color within ROI
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                cv2.putText(frame, "Unregistered", (left, bottom + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (128, 128, 128), 2)
                unregistered_faces.add("Unregistered")

        # Record attendance for unregistered faces if there is motion
        current_time = datetime.now()
        motion_detected = len(contours) > 0
        if motion_detected:
            stationary_counter = 0
            for name in unregistered_faces:
                if name not in unregistered_last_detected or (current_time - unregistered_last_detected[name]) >= timedelta(minutes=1):
                    unregistered_last_detected[name] = current_time
                    record_attendance(name, current_time)
        else:
            stationary_counter += 1
            if stationary_counter >= 30:
                unregistered_faces.clear()

        # Display the frame
        cv2.imshow("Face Recognition", frame)

        # Check for keyboard interrupt
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):  # Check for 'r' key press
            run_registration_program()




    # Release the video capture and destroy windows
    cap.release()
    cv2.destroyAllWindows()

def record_attendance(name, current_time):
    date_str = current_time.strftime("%Y-%m-%d")
    csv_filename = f"{date_str}.csv"

    # Check if the CSV file already exists
    if not os.path.isfile(csv_filename):
        with open(csv_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Time"])

    # Append the attendance entry to the CSV file
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, current_time.strftime("%H:%M:%S")])

if __name__ == "__main__":
    
    run_recognition()
    

Directory 'faces' already exists.


In [4]:
import sys
import os
import face_recognition
import cv2
import dlib
import numpy as np
from datetime import datetime, timedelta
import csv
import subprocess
from scipy.spatial import distance as dist
import fil1

def run_registration_program():
    registration_program = "python3 register.py"  # Replace with the command to run your register.py program
    subprocess.Popen(registration_program, shell=True)

# Function to calculate the eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def run_recognition():
    file_path = '/home/parthjoshi/Desktop/webside/face_detection/Face_recognition/shape_predictor_68_face_landmarks.dat'

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(file_path)

    # Initialize lists for known face encodings and names
    previous_landmarks = None
    liveness_threshold = 5
    known_faces = []
    known_names = []

    # Directory containing face imagescord
    directory = fil1.faces_directory  # Replace with the path to your directory

    # Load known faces and encodings from directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            name = os.path.splitext(filename)[0]
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)

            # Detect face landmarks
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb)
            if len(face_locations) > 0:
                face_location = face_locations[0]
                top, right, bottom, left = face_location

                # Perform face encoding
                face_encoding = face_recognition.face_encodings(rgb, [face_location])[0]

                # Add face encoding and name to the lists
                known_faces.append(face_encoding)
                known_names.append(name)

    # Initialize video capture
    cap = cv2.VideoCapture(0)  # Replace with your video source if not using the webcam

    # Get screen resolution
    screen_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    screen_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # Calculate ROI dimensions
    roi_width = int(screen_width / 2)
    roi_height = int(screen_height / 2)
    roi_x = int((screen_width - roi_width) / 2)
    roi_y = int((screen_height - roi_height) / 2)

    # Initialize tolerance for face recognition (confidence factor)
    tolerance = 0.5  # Adjust this value as needed

    # Initialize dictionary to track last detected time for recognized faces and unregistered faces
    recognized_last_detected = {}
    unregistered_last_detected = {}

    # Create resizable window
    cv2.namedWindow("Face Recognition", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Face Recognition", int(screen_width), int(screen_height))

    # Initialize variables for motion detection
    prev_frame_gray = None
    stationary_counter = 0

    # Initialize blink detection parameters
    EYE_AR_THRESH = 0.2
    EYE_AR_CONSEC_FRAMES = 3
    blink_counter = 0
    blink_detected = False

    # Main loop
    while True:
        # Read a frame
        ret, frame = cap.read()

        # Convert frame to gray for motion detection
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        frame_gray = cv2.GaussianBlur(frame_gray, (21, 21), 0)

        # Initialize previous frame if it's the first iteration
        if prev_frame_gray is None:
            prev_frame_gray = frame_gray
            continue

        # Calculate absolute difference between current and previous frames
        frame_delta = cv2.absdiff(prev_frame_gray, frame_gray)

        # Apply threshold to detect significant intensity differences
        threshold = 30
        _, frame_threshold = cv2.threshold(frame_delta, threshold, 255, cv2.THRESH_BINARY)

        # Dilate the thresholded image to fill in holes
        kernel = np.ones((5, 5), np.uint8)
        frame_threshold = cv2.dilate(frame_threshold, kernel, iterations=2)

        # Find contours of significant differences
        contours, _ = cv2.findContours(frame_threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Convert frame back to RGB for face recognition
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Draw ROI on the frame
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 0, 255), 2)

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(rgb)
        face_encodings = face_recognition.face_encodings(rgb, face_locations)

        # Initialize set to track unregistered face IDs
        unregistered_faces = set()

        # Iterate over the face encodings
        for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
            # Check if the face is completely outside the ROI
            if right < roi_x or left > roi_x + roi_width or bottom < roi_y or top > roi_y + roi_height:
                # Draw bounding box in gray color
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                continue

            # Compare face encodings with known faces
            face_distances = face_recognition.face_distance(known_faces, face_encoding)
            best_match_index = np.argmin(face_distances)
            face_distance = face_distances[best_match_index]

            if face_distance <= tolerance:
                name = known_names[best_match_index]
                current_time = datetime.now()

                if name not in recognized_last_detected or (current_time - recognized_last_detected[name]) >= timedelta(minutes=2):
                    recognized_last_detected[name] = current_time
                    record_attendance(name, current_time)

                # Detect eye landmarks
                landmarks = predictor(rgb, dlib.rectangle(left, top, right, bottom))
                left_eye = np.array([(landmarks.part(36).x, landmarks.part(36).y),
                                     (landmarks.part(37).x, landmarks.part(37).y),
                                     (landmarks.part(38).x, landmarks.part(38).y),
                                     (landmarks.part(39).x, landmarks.part(39).y),
                                     (landmarks.part(40).x, landmarks.part(40).y),
                                     (landmarks.part(41).x, landmarks.part(41).y)], np.int32)
                right_eye = np.array([(landmarks.part(42).x, landmarks.part(42).y),
                                      (landmarks.part(43).x, landmarks.part(43).y),
                                      (landmarks.part(44).x, landmarks.part(44).y),
                                      (landmarks.part(45).x, landmarks.part(45).y),
                                      (landmarks.part(46).x, landmarks.part(46).y),
                                      (landmarks.part(47).x, landmarks.part(47).y)], np.int32)

                left_ear = eye_aspect_ratio(left_eye)
                right_ear = eye_aspect_ratio(right_eye)

                # Average the EAR for both eyes
                ear = (left_ear + right_ear) / 2.0

                # Check if EAR is below the blink threshold
                if ear < EYE_AR_THRESH:
                    blink_counter += 1
                else:
                    if blink_counter >= EYE_AR_CONSEC_FRAMES:
                        blink_detected = True
                    blink_counter = 0

                # Draw bounding box and display name on the frame
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                if blink_detected:
                    cv2.putText(frame, "Blink Detected", (left, top - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    blink_detected = False
            else:
                # Draw bounding box for unknown face in gray color within ROI
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                cv2.putText(frame, "Unregistered", (left, bottom + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (128, 128, 128), 2)
                unregistered_faces.add("Unregistered")

        # Record attendance for unregistered faces if there is motion
        current_time = datetime.now()
        motion_detected = len(contours) > 0
        if motion_detected:
            stationary_counter = 0
            for name in unregistered_faces:
                if name not in unregistered_last_detected or (current_time - unregistered_last_detected[name]) >= timedelta(minutes=1):
                    unregistered_last_detected[name] = current_time
                    record_attendance(name, current_time)
        else:
            stationary_counter += 1
            if stationary_counter >= 30:
                unregistered_faces.clear()

        # Display the frame
        cv2.imshow("Face Recognition", frame)

        # Check for keyboard interrupt
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):  # Check for 'r' key press
            run_registration_program()

    # Release the video capture and destroy windows
    cap.release()
    cv2.destroyAllWindows()

def record_attendance(name, current_time):
    date_str = current_time.strftime("%Y-%m-%d")
    csv_filename = f"{date_str}.csv"

    # Check if the CSV file already exists
    if not os.path.isfile(csv_filename):
        with open(csv_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Time"])

    # Append the attendance entry to the CSV file
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, current_time.strftime("%H:%M:%S")])

if __name__ == "__main__":
    run_recognition()


In [1]:
import sys
import os
import face_recognition
import cv2
import dlib
import numpy as np
from datetime import datetime, timedelta
import csv
import subprocess
import fil1
import os



def run_registration_program():
    registration_program = "python3 register.py"  # Replace with the command to run your register.py program
    subprocess.Popen(registration_program, shell=True)
# Initialize dlib face detector and facial landmark predictor
def run_recognition():
    
    file_path = '/home/parthjoshi/Desktop/webside/face_detection/Face_recognition/shape_predictor_68_face_landmarks.dat'

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(file_path)
   

    # Initialize lists for known face encodings and names
    previous_landmarks = None
    liveness_threshold = 5
    known_faces = []
    known_names = []

    # Directory containing face imagescord
    directory = fil1.faces_directory  # Replace with the path to your directory

    # Load known faces and encodings from directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            name = os.path.splitext(filename)[0]
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)

            # Detect face landmarks
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb)
            if len(face_locations) > 0:
                face_location = face_locations[0]
                top, right, bottom, left = face_location

                # Perform face encoding
                face_encoding = face_recognition.face_encodings(rgb, [face_location])[0]

                # Add face encoding and name to the lists
                known_faces.append(face_encoding)
                known_names.append(name)

    # Initialize video capture
    cap = cv2.VideoCapture(0)  # Replace with your video source if not using the webcam

    # Get screen resolution
    screen_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    screen_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # Calculate ROI dimensions
    roi_width = int(screen_width / 2)
    roi_height = int(screen_height / 2)
    roi_x = int((screen_width - roi_width) / 2)
    roi_y = int((screen_height - roi_height) / 2)

    # Initialize tolerance for face recognition (confidence factor)
    tolerance = 0.5  # Adjust this value as needed

    # Initialize dictionary to track last detected time for recognized faces and unregistered faces
    recognized_last_detected = {}
    unregistered_last_detected = {}

    # Create resizable window
    cv2.namedWindow("Face Recognition", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Face Recognition", int(screen_width), int(screen_height))

    # Initialize variables for motion detection
    prev_frame_gray = None
    stationary_counter = 0

    # Main loop
    while True:
        # Read a frame
        ret, frame = cap.read()

        # Convert frame to gray for motion detection
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        frame_gray = cv2.GaussianBlur(frame_gray, (21, 21), 0)

        # Initialize previous frame if it's the first iteration
        if prev_frame_gray is None:
            prev_frame_gray = frame_gray
            continue

        # Calculate absolute difference between current and previous frames
        frame_delta = cv2.absdiff(prev_frame_gray, frame_gray)

        # Apply threshold to detect significant intensity differences
        threshold = 30
        _, frame_threshold = cv2.threshold(frame_delta, threshold, 255, cv2.THRESH_BINARY)

        # Dilate the thresholded image to fill in holes
        kernel = np.ones((5, 5), np.uint8)
        frame_threshold = cv2.dilate(frame_threshold, kernel, iterations=2)

        # Find contours of significant differences
        contours, _ = cv2.findContours(frame_threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Convert frame back to RGB for face recognition
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Draw ROI on the frame
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 0, 255), 2)

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(rgb)
        face_encodings = face_recognition.face_encodings(rgb, face_locations)

        # Initialize set to track unregistered face IDs
        unregistered_faces = set()

        # Iterate over the face encodings
        for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
            # Check if the face is completely outside the ROI
            if right < roi_x or left > roi_x + roi_width or bottom < roi_y or top > roi_y + roi_height:
                # Draw bounding box in gray color
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                continue

            # Compare face encodings with known faces
            face_distances = face_recognition.face_distance(known_faces, face_encoding)
            best_match_index = np.argmin(face_distances)
            face_distance = face_distances[best_match_index]

            if face_distance <= tolerance:
                name = known_names[best_match_index]
                current_time = datetime.now()

                if name not in recognized_last_detected or (current_time - recognized_last_detected[name]) >= timedelta(minutes=2):
                    recognized_last_detected[name] = current_time
                    record_attendance(name, current_time)
                else:
                    unregistered_faces.add(name)
                
                landmarks = predictor(rgb, dlib.rectangle(left, top, right, bottom))
                

                # Draw bounding box and display name on the frame
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                # Draw bounding box for unknown face in gray color within ROI
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                cv2.putText(frame, "Unregistered", (left, bottom + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (128, 128, 128), 2)
                unregistered_faces.add("Unregistered")

        # Record attendance for unregistered faces if there is motion
        current_time = datetime.now()
        motion_detected = len(contours) > 0
        if motion_detected:
            stationary_counter = 0
            for name in unregistered_faces:
                if name not in unregistered_last_detected or (current_time - unregistered_last_detected[name]) >= timedelta(minutes=1):
                    unregistered_last_detected[name] = current_time
                    record_attendance(name, current_time)
        else:
            stationary_counter += 1
            if stationary_counter >= 30:
                unregistered_faces.clear()

        # Display the frame
        cv2.imshow("Face Recognition", frame)

        # Check for keyboard interrupt
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):  # Check for 'r' key press
            run_registration_program()




    # Release the video capture and destroy windows
    cap.release()
    cv2.destroyAllWindows()

def record_attendance(name, current_time):
    date_str = current_time.strftime("%Y-%m-%d")
    csv_filename = f"{date_str}.csv"

    # Check if the CSV file already exists
    if not os.path.isfile(csv_filename):
        with open(csv_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Time"])

    # Append the attendance entry to the CSV file
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, current_time.strftime("%H:%M:%S")])

if __name__ == "__main__":
    run_recognition()

Directory 'faces' already exists.


In [2]:
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox
import subprocess
import cv2
import os

class FaceRecognitionUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition Application")

        # Initialize variables for camera selection
        self.cameras = self.get_available_cameras()
        self.selected_camera = tk.StringVar()
        self.selected_camera.set(self.cameras[0] if self.cameras else "")

        # Create main frame to hold everything
        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Create frame for face recognition display
        self.recognition_frame = tk.Frame(self.main_frame, bd=2, relief=tk.SUNKEN)
        self.recognition_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Placeholder for face recognition display (you can replace this with actual recognition display logic)
        self.recognition_label = tk.Label(self.recognition_frame, text="Face Recognition Display", font=("Arial", 24))
        self.recognition_label.pack(pady=50, padx=50, expand=True)

        # Create frame for control buttons
        self.control_frame = tk.Frame(self.main_frame, bd=2, relief=tk.RAISED)
        self.control_frame.pack(side=tk.RIGHT, fill=tk.Y)

        # Add buttons to the control frame
        self.registration_button = ttk.Button(self.control_frame, text="Registration", command=self.open_registration)
        self.registration_button.pack(pady=20, padx=10, fill=tk.X)

        self.camera_button = ttk.Button(self.control_frame, text="Camera Options", command=self.show_camera_options)
        self.camera_button.pack(pady=20, padx=10, fill=tk.X)

        self.stop_button = ttk.Button(self.control_frame, text="Stop", command=self.stop_recognition)
        self.stop_button.pack(pady=20, padx=10, fill=tk.X)

        # Start the face recognition process
        self.start_recognition_process()

    def start_recognition_process(self):
        # Replace with your actual command to run the Application.py
        self.recognition_process = subprocess.Popen(["python3", "Application.py"])

    def open_registration(self):
        # Replace with your actual command to run the registration code
        subprocess.Popen(["python3", "register.py"])

    def get_available_cameras(self):
        # Function to detect available cameras
        available_cameras = []
        for i in range(10):  # Assume up to 10 cameras
            cap = cv2.VideoCapture(i)
            if cap.isOpened():
                available_cameras.append(f"Camera {i}")
                cap.release()
        return available_cameras

    def show_camera_options(self):
        if not self.cameras:
            messagebox.showinfo("No Cameras", "No cameras detected.")
            return

        camera_dialog = tk.Toplevel(self.root)
        camera_dialog.title("Camera Options")

        camera_label = tk.Label(camera_dialog, text="Select Camera:")
        camera_label.pack(pady=10)

        camera_dropdown = ttk.Combobox(camera_dialog, textvariable=self.selected_camera, values=self.cameras, state="readonly")
        camera_dropdown.pack(pady=10)

        ok_button = ttk.Button(camera_dialog, text="OK", command=camera_dialog.destroy)
        ok_button.pack(pady=10)

    def stop_recognition(self):
        if hasattr(self, 'recognition_process') and self.recognition_process.poll() is None:
            self.recognition_process.terminate()
            messagebox.showinfo("Stopped", "Face Recognition process stopped.")
        else:
            messagebox.showinfo("Not Running", "Face Recognition process is not running.")

if __name__ == "__main__":
    root = tk.Tk()
    app = FaceRecognitionUI(root)
    root.mainloop()


[ WARN:0@28.930] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@28.931] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@28.931] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@28.931] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@28.931] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@28.932] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@28.932] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@28.932] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@28.932] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@28.933] global obsensor_uvc_stream_channel.cpp:159 getStreamChan

Directory 'faces' already exists.


In [3]:
import sys
import os
import face_recognition
import cv2
import dlib
import numpy as np
from datetime import datetime, timedelta
import csv
import subprocess
from tkinter import Tk, Label, Button, filedialog, messagebox

def run_registration_program():
    """Launches the registration program using a tkinter file dialog."""
    root = Tk()
    root.withdraw()  # Hide the main window

    registration_program_path = filedialog.askopenfilename(
        title="Select Registration Program",
        filetypes=[("Python Files", "*.py")])

    if registration_program_path:
        subprocess.Popen(["python3", registration_program_path])
    else:
        messagebox.showerror("Error", "No registration program selected.")

def run_recognition():
    """Performs face recognition with a user-friendly UI."""

    def update_camera_selection():
        """Updates the selected camera index based on user input."""
        global camera_index
        camera_index = camera_selection_var.get()
        start_button.config(state="normal")  # Enable start button after selection

    def start_recognition():
        """Starts the face recognition process."""
        stop_button.config(state="normal")  # Enable stop button
        start_button.config(state="disabled")  # Disable start button

        # Initialize dlib face detector and facial landmark predictor
        file_path = '/home/parthjoshi/Desktop/webside/face_detection/Face_recognition/shape_predictor_68_face_landmarks.dat'
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(file_path)

        # Initialize lists for known face encodings and names
        previous_landmarks = None
        liveness_threshold = 5
        known_faces = []
        known_names = []

        # Directory containing face images
        directory = fil1.faces_directory  # Replace with the path to your directory

        # Load known faces and encodings from directory
        for filename in os.listdir(directory):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                name = os.path.splitext(filename)[0]
                image_path = os.path.join(directory, filename)
                image = cv2.imread(image_path)

                # Detect face landmarks
                rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                face_locations = face_recognition.face_locations(rgb)
                if len(face_locations) > 0:
                    face_location = face_locations[0]
                    top, right, bottom, left = face_location

                    # Perform face encoding
                    face_encoding = face_recognition.face_encodings(rgb, [face_location])[0]

                    # Add face encoding and name to the lists
                    known_faces.append(face_encoding)
                    known_names.append(name)

        # Initialize video capture
        cap = cv2.VideoCapture(camera_index)

        # Get screen resolution
        screen_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        screen_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

        # Calculate ROI dimensions
        roi_width = int(screen_width / 2)
        roi_height = int(screen_height / 2)
        roi_x = int((screen_width - roi_width) / 2)
        roi_y = int((screen_height - roi_height) / 2)

        # Initialize tolerance for face recognition (confidence factor)
        tolerance = 0.7  # Adjust this value as needed

        # Initialize dictionary to track last detected time for recognized faces and unregistered faces
        recognized_last_detected = {}
        unregistered_last_detected = {}

        # Create resizable window
        window_name = "Face Recognition"
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(window_name, int(screen_width), int(screen_height))

        # Initialize variables for motion detection
        prev_frame_gray = None
        stationary_counter = 0

        # Main loop
    while True:
        # Read a frame
        ret, frame = cap.read()

        # Convert frame to gray for motion detection
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        frame_gray = cv2.GaussianBlur(frame_gray, (21, 21), 0)

        # Initialize previous frame if it's the first iteration
        if prev_frame_gray is None:
            prev_frame_gray = frame_gray
            continue

        # Calculate absolute difference between current and previous frames
        frame_delta = cv2.absdiff(prev_frame_gray, frame_gray)

        # Apply threshold to detect significant intensity differences
        threshold = 30
        _, frame_threshold = cv2.threshold(frame_delta, threshold, 255, cv2.THRESH_BINARY)

        # Dilate the thresholded image to fill in holes
        kernel = np.ones((5, 5), np.uint8)
        frame_threshold = cv2.dilate(frame_threshold, kernel, iterations=2)

        # Find contours of significant differences
        contours, _ = cv2.findContours(frame_threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Convert frame back to RGB for face recognition
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Draw ROI on the frame
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 0, 255), 2)

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(rgb)
        face_encodings = face_recognition.face_encodings(rgb, face_locations)

        # Initialize set to track unregistered face IDs
        unregistered_faces = set()

        # Iterate over the face encodings
        for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
            # Check if the face is completely outside the ROI
            if right < roi_x or left > roi_x + roi_width or bottom < roi_y or top > roi_y + roi_height:
                # Draw bounding box in gray color
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                continue

            # Compare face encodings with known faces
            face_distances = face_recognition.face_distance(known_faces, face_encoding)
            best_match_index = np.argmin(face_distances)
            face_distance = face_distances[best_match_index]

            if face_distance <= tolerance:
                name = known_names[best_match_index]
                current_time = datetime.now()

                if name not in recognized_last_detected or (current_time - recognized_last_detected[name]) >= timedelta(minutes=2):
                    recognized_last_detected[name] = current_time
                    record_attendance(name, current_time)
                else:
                    unregistered_faces.add(name)
                
                landmarks = predictor(rgb, dlib.rectangle(left, top, right, bottom))
                

                # Draw bounding box and display name on the frame
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                # Draw bounding box for unknown face in gray color within ROI
                cv2.rectangle(frame, (left, top), (right, bottom), (128, 128, 128), 2)
                cv2.putText(frame, "Unregistered", (left, bottom + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (128, 128, 128), 2)
                unregistered_faces.add("Unregistered")

        # Record attendance for unregistered faces if there is motion
        current_time = datetime.now()
        motion_detected = len(contours) > 0
        if motion_detected:
            stationary_counter = 0
            for name in unregistered_faces:
                if name not in unregistered_last_detected or (current_time - unregistered_last_detected[name]) >= timedelta(minutes=1):
                    unregistered_last_detected[name] = current_time
                    record_attendance(name, current_time)
        else:
            stationary_counter += 1
            if stationary_counter >= 30:
                unregistered_faces.clear()

        # Display the frame
        cv2.imshow("Face Recognition", frame)

        # Check for keyboard interrupt
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):  # Check for 'r' key press
            run_registration_program()




    # Release the video capture and destroy windows
    cap.release()
    cv2.destroyAllWindows()

def record_attendance(name, current_time):
    date_str = current_time.strftime("%Y-%m-%d")
    csv_filename = f"{date_str}.csv"

    # Check if the CSV file already exists
    if not os.path.isfile(csv_filename):
        with open(csv_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Time"])

    # Append the attendance entry to the CSV file
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, current_time.strftime("%H:%M:%S")])

if __name__ == "__main__":
    run_recognition()
        


NameError: name 'cap' is not defined